## Assignment 6 Neural Networks with Keras and TensorFlow:
Student name | Hours spent on the tasks
------------ | -------------
Lenia Malki | 12
Maële Belmont | 12

Download the notebook for this assignment. The notebook will provide a basis that you can
use to solve the exercises.
This assignment will work with the MNIST data set. The MNIST dataset is a standard
benchmark where small 28x28 pixel grayscale images of handwritten digits. Each image was
manually assigned to a class label, an integer from 0 to 9, by the US Census Bureau. The task
associated with the dataset is building a model that takes a new image (of the same size)
and returns a class label – that is, an integer from 0 to 9.
For this assignment, we will use the Keras framework to construct our neural networks. You
can read more about the Keras framework here https://keras.io/. You can find information
regarding the different layers and regularizers there. In this assignment, you can use free
GPU leases on Google Colab or deepnote to speed up training.

## What to submit:
•All Python code written.

• A report that includes the figures produced and the descriptions/discussions that are requested in the questions.

If you upload a zip file, please also upload any PDF files separately (so that they can be viewed more
conveniently in Canvas).

### Self-check
Is all the required information on the front page? Have you answered all questions to the best of your ability? Anything else you can easily check? (details, terminology, arguments, clearly stated
answers etc.?)  

### Grading
Grading will be based on a qualitative assessment of each assignment. It is important to:

• Present clear arguments
• Present the results in a pedagogical way
• Should it be table/plot? What kind of plot? Is everything clear and easy to
understand?
• Show understanding of the topics
• Give correct solutions.
• Make sure that the code is well commented.
• Important parts of the code should be included in the running text and the full
code uploaded to Canvas. 

### Code

In [6]:
# imports
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import tensorflow as tf
from matplotlib import pyplot as plt
from keras.utils import np_utils

In [7]:
# Hyper-parameters data-loading and formatting

batch_size = 128
num_classes = 10
epochs = 10

img_rows, img_cols = 28, 28

(x_train, lbl_train), (x_test, lbl_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

**Preprocessing**

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

y_train = keras.utils.np_utils.to_categorical(lbl_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(lbl_test, num_classes)

In [8]:
## Define model ##
model = Sequential()

model.add(Flatten())
model.add(Dense(64, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(num_classes, activation='softmax'))


model.compile(loss=keras.losses.categorical_crossentropy,
               optimizer=tf.keras.optimizers.SGD(learning_rate = 0.1),
        metrics=['accuracy'],)

fit_info = model.fit(x_train, y_train,
           batch_size=batch_size,
           epochs=epochs,
           verbose=1,
           validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss: {}, Test accuracy {}'.format(score[0], score[1]))

model.summary()

Epoch 1/10
469/469 [==============================] - 2s 3ms/step - loss: 119048765440.0000 - accuracy: 0.1114 - val_loss: 2.3012 - val_accuracy: 0.1135
Epoch 2/10
469/469 [==============================] - 1s 2ms/step - loss: 2.3014 - accuracy: 0.1122 - val_loss: 2.3012 - val_accuracy: 0.1135
Epoch 3/10
469/469 [==============================] - 1s 2ms/step - loss: 2.3014 - accuracy: 0.1124 - val_loss: 2.3010 - val_accuracy: 0.1135
Epoch 4/10
469/469 [==============================] - 1s 2ms/step - loss: 2.3014 - accuracy: 0.1124 - val_loss: 2.3012 - val_accuracy: 0.1135
Epoch 5/10
469/469 [==============================] - 1s 2ms/step - loss: 2.3014 - accuracy: 0.1124 - val_loss: 2.3010 - val_accuracy: 0.1135
Epoch 6/10
469/469 [==============================] - 1s 2ms/step - loss: 2.3013 - accuracy: 0.1124 - val_loss: 2.3013 - val_accuracy: 0.1135
Epoch 7/10
469/469 [==============================] - 1s 2ms/step - loss: 2.3014 - accuracy: 0.1116 - val_loss: 2.3010 - val_accuracy: 0.

**Question 4 Auto-Encoder for denoising**

In [9]:
import numpy as np
def salt_and_pepper(input, noise_level=0.5):
    """
    This applies salt and pepper noise to the input tensor - randomly setting bits to 1 or 0.
    Parameters
    ----------
    input : tensor
        The tensor to apply salt and pepper noise to.
    noise_level : float
        The amount of salt and pepper noise to add.
    Returns
    -------
    tensor
        Tensor with salt and pepper noise applied.
    """
    # salt and pepper noise
    a = np.random.binomial(size=input.shape, n=1, p=(1 - noise_level))
    b = np.random.binomial(size=input.shape, n=1, p=0.5)
    c = (a==0) * b
    return input * a + c


#data preparation
flattened_x_train = x_train.reshape(-1,784)
flattened_x_train_seasoned = salt_and_pepper(flattened_x_train, noise_level=0.4)

flattened_x_test = x_test.reshape(-1,784)
flattened_x_test_seasoneed = salt_and_pepper(flattened_x_test, noise_level=0.4)

In [10]:
latent_dim = 96  

input_image = keras.Input(shape=(784,))
encoded = Dense(128, activation='relu')(input_image)
encoded = Dense(latent_dim, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(encoded)
decoded = Dense(784, activation='sigmoid')(decoded)

autoencoder = keras.Model(input_image, decoded)
encoder_only = keras.Model(input_image, encoded)

encoded_input = keras.Input(shape=(latent_dim,))
decoder_layer = Sequential(autoencoder.layers[-2:])
decoder = keras.Model(encoded_input, decoder_layer(encoded_input))

autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [11]:
fit_info_AE = autoencoder.fit(flattened_x_train_seasoned, flattened_x_train,
                epochs=32,
                batch_size=64,
                shuffle=True,
                validation_data=(flattened_x_test_seasoneed, flattened_x_test))

Epoch 1/32
938/938 [==============================] - 5s 5ms/step - loss: -1945588596736.0000 - val_loss: -10144049004544.0000
Epoch 2/32
938/938 [==============================] - 4s 5ms/step - loss: -53970793922560.0000 - val_loss: -134618351140864.0000
Epoch 3/32
938/938 [==============================] - 4s 4ms/step - loss: -299186884968448.0000 - val_loss: -545028799725568.0000
Epoch 4/32
938/938 [==============================] - 4s 4ms/step - loss: -901576851980288.0000 - val_loss: -1405418928603136.0000
Epoch 5/32
938/938 [==============================] - 4s 4ms/step - loss: -2017166018740224.0000 - val_loss: -2873693898276864.0000
Epoch 6/32
938/938 [==============================] - 4s 4ms/step - loss: -3798908505423872.0000 - val_loss: -5111708477554688.0000
Epoch 7/32
938/938 [==============================] - 4s 4ms/step - loss: -6412637965385728.0000 - val_loss: -8298006201761792.0000
Epoch 8/32
938/938 [==============================] - 4s 4ms/step - loss: -100298009031

### 1. Preprocessing (1p)

In the notebook, the data is downloaded from an external server imported into the notebook
environment using the mnist.load_data() function call.
Explain the data pre-processing high-lighted in the notebook.
- explain the data split between train and test? https://www.tensorflow.org/api_docs/python/tf/keras/datasets/mnist/load_data

### 2. Network model, training, and changing hyper-parameters. (4p)

**2(A)**
How many layers does the network in the notebook have? How many neurons does each layer
have? What activation functions and why are these appropriate for this application? What is the
total number of parameters for the network? Why does the input and output layers have the
dimensions they have? <br>


- The network in the notebook has 4 layers:

n° | Layer name | Neurons in layer  
------------ | ------------ | ------------- 
1 | flatten_3 | 784 
2 | dense_12 | 64 
3 | dense_13 | 64 
4 | dense_14 | 10 

- The activation functions are 'relu' and 'softmax'. These are appropriate for this application [because](https://keras.io/api/layers/activations/) ... .
- The total number of parameters for the network is 55,050.
- The input and output layers have these dimensions because ... .

##### 2(B)
What loss-function is used to train the network? What is the functional form (mathematical
expression) of the loss function? and how should we interpret it? Why is it appropriate for the
problem at hand?
- Categorical cross entropy loss function
- Functional form of the loss function: ...

##### 2(C)
Train the network for 10 epochs and plot the training and validation accuracy for each epoch

##### 2(D)
Update model to implement a three-layer neural network where the hidden-layers has 500
and 300 hidden units respectively. Train for 40 epochs. What is the best validation accuracy you
can achieve? – Geoff Hinton (a co-pioneer of Deep learning) claimed this network could reach a
validation accuracy of 0.9847 (http://yann.lecun.com/exdb/mnist/) using weight decay (L2
regularization of weights (kernels): https://keras.io/api/layers/regularizers/). Implement weight
decay on hidden units and train and select 5 regularization factors from 0.000001 to 0.001. Train
3 replicates networks for each regularization factor. Plot the final validation accuracy with
standard deviation (computed from the replicates) as a function of the regularization factor. How
close do you get to Hintons result? – If you do not get the same results, what factors may influence this? (hint: What information is not given by Hinton on the MNIST database that may
influence Model training) 

### 3.  Convolutional layers. (2p)

##### 3(A)
Design a model that makes use of at least one convolutional layer – how performant a model can
you get? -- According to the MNIST database it should be possible reach to 99% accuracy on the
validation data. If you choose to use any layers apart from convolutional layers and layers that you
used in previous questions, you must describe what they do. If you do not reach 99% accuracy,
report your best performance and explain your attempts and thought process.

##### 3(B)
Discuss the differences and potential benefits of using convolutional layers over fully connected
ones for the particular application?

### 4. Auto-Encoders for denoising (3p)

##### 4(A)
The notebook implements a simple denoising deep autoencoder model. Explain what the model
does: use the data-preparation and model definition code to explain how the goal of the model is
achieved. Explain the role of the loss function? Draw a diagram of the model and include it in your
report. Train the model with the settings given.

##### 4(B)
Add increasing levels of noise to the test-set using the salt_and_pepper()-function (0 to 1).
Use matplotlib to visualize a few examples (3-4) in the original, “seasoned” (noisy), and denoised
versions (Hint: for visualization use imshow(), use the trained autoencoder to denoise the noisy
digits). At what noise level does it become difficult to identify the digits for you? At what noise level
does the denoising stop working?

##### 4(C)
Test whether denoising improves the classification with the best performing model you obtained
in questions 2 or 3. Plot the true-positive rate as a function of noise-level for the seasoned and
denoised datasets – assume that the correct classification is the most likely class-label. Discuss your
results. 

##### 4(D)
Explain how you can use the decoder part of the denoising auto-encoder to generate synthetic
“hand-written” digits? – Describe the procedure and show examples in your report.